In [2]:
import pandas as pd
import numpy as np
import scipy as sp
import os
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_context("notebook")

DATA_DIR = 'data'

#### Get the data from csv files that are produced by the python scripts we run on the cluster

In [163]:
# load the mean tone a country has about itself
country_to_country_view = pd.read_csv(os.path.join(DATA_DIR, 'country_to_country_view.csv')).dropna()  # 3 rows have no mention_country
country_inner_view = pd.read_csv(os.path.join(DATA_DIR, 'country_inner_view.csv'))
country_outer_view = pd.read_csv(os.path.join(DATA_DIR, 'country_outer_view.csv'))

In [168]:
# iteratively decreasing the weight of a country with bad reputation to compute the reputation
# of another country
c_to_c_transition = country_to_country_view[["actor_country", "mention_country", "avg_tone", 
                                              "count", "avg_weighted_tone"]]
# other columns cannot be splitted from country to country

c_to_c_transition = c_to_c_transition[c_to_c_transition.actor_country !=
                                             c_to_c_transition.mention_country]


# get the count of articles for each actor_country and weight avg_tone and avg_weighted_tone by 
# this, i.e. avg_tone = avg_tone * count / (sum count for actor_country)
counts = c_to_c_transition.groupby("actor_country")["count"].agg("sum")
c_to_c_transition = c_to_c_transition.merge(counts.to_frame(), left_on="actor_country",
                                            right_on="actor_country")
c_to_c_transition.avg_tone = c_to_c_transition.avg_tone * c_to_c_transition.count_x / \
                            c_to_c_transition.count_y
c_to_c_transition.avg_weighted_tone = c_to_c_transition.avg_weighted_tone * \
                            c_to_c_transition.count_x / c_to_c_transition.count_y
c_to_c_transition = c_to_c_transition.drop(["count_x", "count_y"], axis=1)


# putting the country to country transitions in the form of a np matrix that will be more
# suitable for multiplication
index_to_country = np.unique(country_to_country_view.actor_country.values)
country_to_index = {country: index for index, country in enumerate(index_to_country)}

transition_matrix = np.zeros((countries_index.shape[0], countries_index.shape[0]))
for row in c_to_c_transition.itertuples(index=False):
    if row.mention_country in country_to_index:  # we do not take the country that don't have
        # mentions from other countries into account
        transition_matrix[country_to_index[row.actor_country],
                      country_to_index[row.mention_country]] = row.avg_tone

prev_scores = np.zeros((countries_index.shape[0], 1))  # countries scores, set to zeros so that
# we enter the loop
scores = np.ones((countries_index.shape[0], 1)) / countries_index.shape[0]
while(np.linalg.norm(scores - prev_scores) > 0.01):
    print(np.linalg.norm(scores - prev_scores))
    prev_scores = scores / np.linalg.norm(scores)  # scores != 0
    scores = np.matmul(transition_matrix, prev_scores)

14.177446878757825
3.803199725420452
3.4814170571109453
3.459624755390809
3.4524991752017935
3.4555655631603
3.4538889629012917
3.454906559247915
3.454304122143562
3.4546656107006597
3.454447036016
3.4545785620942557
3.4544990093542807
3.454546995835436
3.4545179974372835
3.454535504977812
3.4545249291840285
3.454531315945989
3.454527458367218
3.4545297881484616
3.4545283810205962
3.454529230871498
3.454528717588607
3.454529027593346
3.4545288403608683
3.454528953442837
3.4545288851451668
3.454528926394624
3.454528901481357
3.4545289165281194
3.4545289074403884
3.454528912929068
3.4545289096140928
3.454528911616224
3.4545289104070056
3.4545289111373325
3.45452891069624
3.4545289109626443
3.454528910801746
3.4545289108989228
3.454528910840231
3.4545289108756787
3.454528910854269
3.4545289108672
3.45452891085939
3.4545289108641075
3.4545289108612587
3.454528910862979
3.45452891086194
3.454528910862568
3.454528910862188
3.4545289108624178
3.454528910862279
3.4545289108623622
3.45452891086

3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.454528910862331
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.454528910862331
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.454528910862331
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.454528910862331
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.454528910862331
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.454528910862331
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.454528910862331
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307


3.454528910862331
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.454528910862331
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.454528910862331
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.454528910862331
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.454528910862331
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.454528910862331
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.454528910862331
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.454528910862331
3.4545289108623307
3.4545289108623307
3.4545289108623307
3

3.454528910862331
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.454528910862331
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.454528910862331
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.454528910862331
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.454528910862331
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.454528910862331
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.454528910862331
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.454528910862331
3.4545289108623307
3.4545289108623307
3.4545289108623307
3

3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.454528910862331
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.454528910862331
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.454528910862331
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.454528910862331
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.454528910862331
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.454528910862331
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.454528910862331
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307


...

3.4545289108623307
3.454528910862331
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.454528910862331
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.454528910862331
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.454528910862331
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.454528910862331
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.454528910862331
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.454528910862331
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.454528910862331
3.4545289108623307
3.4545289108623307
3

3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.454528910862331
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.454528910862331
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.454528910862331
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.454528910862331
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.454528910862331
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.454528910862331
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.454528910862331
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307


3.4545289108623307
3.4545289108623307
3.454528910862331
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.454528910862331
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.454528910862331
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.454528910862331
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.454528910862331
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.454528910862331
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.454528910862331
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.454528910862331
3.4545289108623307
3

3.4545289108623307
3.4545289108623307
3.454528910862331
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.454528910862331
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.454528910862331
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.454528910862331
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.454528910862331
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.454528910862331
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.454528910862331
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.4545289108623307
3.454528910862331
3.4545289108623307
3

KeyboardInterrupt: 

,actor_country,mention_country,avg_tone,stddev_tone,count,avg_conf,avg_weighted_tone,std_weighted_tone,first_quartile_tone,median_tone,third_quartile_tone
